In [ ]:
# Подключение и монтирование диска в GoogleColab
from google.colab import drive
drive.mount('/content/drive/')

import os
os.chdir("/content/drive/My Drive/Signature")


os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="0"
from tensorflow.python.client import device_lib

In [ ]:
# Импорт необходимых библиотек 
from keras.preprocessing.image import ImageDataGenerator
from keras import models
from keras import layers
import PIL
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import skimage

In [ ]:
# Глобальные переменные 
train_dir = 'data/train'
val_dir = 'data/val'
test_dir = 'data/test'
img_width = 200
img_height = 100
input_shape = (img_height,img_width,3)
batch_size = 50
target_size =  (img_height,img_width)

In [ ]:
from keras.applications import VGG16
conv_base = VGG16(weights = 'imagenet',
                  include_top = False,
                  input_shape = input_shape)

In [ ]:
model = models.Sequential()
model.add(conv_base)
# До бавление нового классификатора
model.add(layers.Flatten())
model.add(layers.Dense(1024,activation='relu'))
model.add(layers.Dense(50,activation='softmax'))
# Фиксирование свёрточной основы
conv_base.trainable= False 

In [ ]:
# Компиляция модели
model.compile(optimizer='rmsprop', # Алгоритм обратного распространения ошибки
             loss='categorical_crossentropy', # Функция потерь - перекрестная энтропия
             metrics = ['accuracy'])

In [ ]:
# Функция предобработки
def function(image):
  #отсечение фона
  image[:][np.mean(image[:][:],axis=2)>240] = 255
  return image

In [ ]:
# Датагены для обучения и проверки 
train_datagen = ImageDataGenerator(
                                  rescale=1. / 255, # Масштабирование
                                  rotation_range = 5, # Поворот
                                  zoom_range=[1.3, 1.4],                                                                   
                                  width_shift_range= [0,20], # Сдвиг по горизонтали 
                                  height_shift_range= [0,10], # Сдвиг по вертикали 
                                  preprocessing_function = function, # Функция предобработки
                                  cval = 255.0, # Значения заполнения
                                  fill_mode = 'constant' # Метод заполнения
                                   )
test_datagen = ImageDataGenerator(
                                  rescale=1. / 255,  
                                  zoom_range=[1.35, 1.35],
                                  preprocessing_function = function,
                                  cval = 255.0,
                                  fill_mode = 'constant'
                                   )

In [ ]:
# Генераторы для обученя, проверки, и тестирования
train_generator = train_datagen.flow_from_directory(
    train_dir, 
    target_size = target_size,
    batch_size = batch_size,
    color_mode="grayscale", # Вид изображения 
    class_mode='categorical') 

val_generator = test_datagen.flow_from_directory( 
    val_dir,
    target_size = target_size,
    batch_size = batch_size,
    color_mode="grayscale",
    class_mode='categorical')

test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size = target_size,
    batch_size = batch_size,
    color_mode="grayscale",
    class_mode='categorical',
    )

In [ ]:
# Метод для преждевременной остановки обучения
from  keras.callbacks import Callback
class EarlyStoppingByAccuracy(Callback):
    def __init__(self, monitor='accuracy', value=1.00, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose
    def on_epoch_end(self, epoch, logs={}):
      current = logs.get(self.monitor)
      if current is None:
          warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

      if current >= self.value:
          if self.verbose > 0:
              print("Epoch %05d: early stopping THR" % epoch)
          self.model.stop_training = True

# Обучение модели
from PIL import  Image
import keras.callbacks
history = model.fit_generator(
    train_generator,
    callbacks = [ EarlyStoppingByAccuracy(monitor='val_accuracy', value=0.99, verbose=2)],
    steps_per_epoch = 500 // batch_size,
    epochs=50,
    validation_data= val_generator,
    validation_steps = 5)

In [ ]:
# Графики процесса обучения
plt.style.use('ggplot')
def plot_history(history):
    acc = history.history['accuracy']
    val_acc = history.history['val_accuracy']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 4))
    plt.subplot(1, 2, 1)
    plt.plot(x, (acc), 'b', label='Точность на тренировочном наборе')
    plt.plot(x, (val_acc), 'r', label='Точность на валидационном наборе')
    plt.title('Тренировочная и валидационная точность')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, (loss), 'b', label='Потери при обучении')
    plt.plot(x, (val_loss), 'r', label='Потери при валидации')
    plt.title('Тренировочные и валидационные потери')
    plt.legend()
plot_history(history)    
#Точность на тестовых 
scores = model.evaluate_generator(test_generator, 100//20)
print("Аккуратность на тестовых данных: %.2f%%" % (scores[1]*100))

In [ ]:
# Метод mypredict для идентификации подписи
def mypredict(img_path):
  img = image.load_img(img_path,target_size = target_size)
  x = image.img_to_array(img)
  x = x.reshape((1,) + x.shape)
  i = 0 
  for batch in preproc_datagen.flow(x,batch_size=1):
    imgplot = plt.imshow(image.array_to_img(batch[0]),cmap = 'Greys')
    print("Точность: "+str(model.predict(batch).max()))
    print("Класс: "+str(model.predict_classes(batch)))
    
    i += 1
    if i%1==0:
      break
  plt.show

mypredict(fnames[1])